#### Import classes

In [1]:
from impl import JournalUploadHandler, CategoryUploadHandler, JournalQueryHandler, CategoryQueryHandler, BasicQueryEngine
path = 'http://192.168.0.205:9999/blazegraph'
grp_endpoint = f'{path}/sparql'
# java -server -Xmx1g -jar blazegraph.jar

# set Journal methods

jou = JournalUploadHandler()
jou.setDbPathOrUrl(grp_endpoint)
jou.pushDataToDb('test_data/doaj.csv')

jou_qh = JournalQueryHandler()
jou_qh.setDbPathOrUrl(grp_endpoint)

# set Category methods

real_path = 'relational.db'
cat = CategoryUploadHandler()
cat.setDbPathOrUrl(real_path)
cat.pushDataToDb('test_data/scimago.json')

cat_qh = CategoryQueryHandler()
cat_qh.setDbPathOrUrl(real_path)

# set Engine methods

que = BasicQueryEngine()
que.addJournalHandler(jou_qh)

True

#### Check JournalQueryHandler

In [3]:
# result_q1 = jou_qh.getJournalsWithTitle('مجلة كلي')
# result_q1 = jou_qh.getAllJournals()
# result_q1 = jou_qh.getJournalsWithDOAJSeal()
result_q1 = jou_qh.getById('2532-2615') # 2532-2615 2385-2615

result_q1

SPARQL Error: <urlopen error [WinError 10061] No connection could be made because the target machine actively refused it>


""


#### Check CategoryQueryHandler

In [2]:
result_q1 = cat_qh.getCategoriesAssignedToAreas(set())

# a = "Q1"
result_q1
# print(', '.join(['?'] * len(a)))

DatabaseError: Execution failed on sql '
                    SELECT DISTINCT c.id, a.area
                    FROM categories c 
                ': no such column: a.area

### Check QueryEngine

In [7]:
# result_q1 = que.getJournalsWithTitle('مجلة كلي')
result_q1 = que.getAllJournals()
# reslt_q1 = que.getJournalsWithDOAJSeal()
result_q1

([<impl.Journal at 0x116d2eba550>,
 198)

#### edo stuff

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from sqlite3 import connect
import SPARQLWrapper
from rdflib import Graph, URIRef, Literal, RDF 
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore

In [ ]:
df = pd.read_csv('test_data/doaj.csv')
counter = 0
# df.info()
entrambi_nan_1 = df[['Journal ISSN (print version)', 'Journal EISSN (online version)']].isna().all(axis=1).sum()
print(entrambi_nan_1)

entrambi_non_nan = ~(df['Journal ISSN (print version)'].isna()) | ~(df['Journal EISSN (online version)'].isna())

# Conta il numero di volte in cui la condizione è True
conteggio = entrambi_non_nan.sum()
print(conteggio)

seal_count = (df['DOAJ Seal'] == 'Yes').sum()
print(seal_count)

0
198
11


In [ ]:
    def getCategoriesWithQuartile(self, quartiles: set):
        with connect(self.dbPathOrUrl) as con:
            if not quartiles:
                # Se l'insieme dei quartili in input è vuoto, seleziona tutte le categorie
                query = """
                    SELECT DISTINCT category_internal_id, id, quartile
                    FROM categories
                """
                df = pd.read_sql_query(query, con)
            else:
                # Altrimenti, seleziona solo le categorie con i quartili specificati
                placeholders = ', '.join(['?'] * len(quartiles))
                query = f"""
                    SELECT DISTINCT category_internal_id, id, quartile
                    FROM categories
                    WHERE quartile IN ({placeholders})
                """
                df = pd.read_sql_query(query, con, params=list(quartiles))
        return df

    # Prendere tutte le categorie associate a una lista di aree
    def getCategoriesAssignedToAreas(self, areas: set):
        with connect(self.dbPathOrUrl) as con:
            if not areas:
                query = f"""
                    SELECT DISTINCT c.id, a.area
                    FROM categories c 
                """
                df = pd.read_sql_query(query, con)
            else:
                placeholders = ', '.join(['?'] * len(areas))
                query = f"""
                SELECT DISTINCT c.id, a.area
                FROM categories c
                JOIN areas a ON c.item_internal_id = a.item_internal_id
                WHERE a.area IN ({placeholders})
                """
                df = pd.read_sql_query(query, con, params=list(areas))
        return df